In [2]:
import numpy as np
import pandas as pd
import seaborn as sns

df = pd.read_csv('smoke_detection_iot.csv')
# print(df)
display(df)

,Unnamed: 0,UTC,Temperature[C],Humidity[%],TVOC[ppb],eCO2[ppm],Raw H2,Raw Ethanol,Pressure[hPa],PM1.0,PM2.5,NC0.5,NC1.0,NC2.5,CNT,Fire Alarm
0,0,1654733331,20.000,57.36,0,400,12306,18520,939.735,0.00,0.00,0.00,0.000,0.000,0,0
1,1,1654733332,20.015,56.67,0,400,12345,18651,939.744,0.00,0.00,0.00,0.000,0.000,1,0
2,2,1654733333,20.029,55.96,0,400,12374,18764,939.738,0.00,0.00,0.00,0.000,0.000,2,0
3,3,1654733334,20.044,55.28,0,400,12390,18849,939.736,0.00,0.00,0.00,0.000,0.000,3,0
4,4,1654733335,20.059,54.69,0,400,12403,18921,939.744,0.00,0.00,0.00,0.000,0.000,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62625,62625,1655130047,18.438,15.79,625,400,13723,20569,936.670,0.63,0.65,4.32,0.673,0.015,5739,0
62626,62626,1655130048,18.653,15.87,612,400,13731,20588,936.678,0.61,0.63,4.18,0.652,0.015,5740,0
62627,62627,1655130049,18.867,15.84,627,400,13725,20582,936.687,0.57,0.60,3.95,0.617,0.014,5741,0
62628,62628,1655130050,19.083,16.04,638,400,13712,20566,936.680,0.57,0.59,3.92,0.611,0.014,5742,0


In [3]:
df.shape

(62630, 16)

In [4]:
df["Fire Alarm"].value_counts()
# prefer to F1-score than validation score (accuracy) because the number of examples in both class labels are not so equal distributed
# but still considered both in process? -> ask tutor

1    44757
0    17873
Name: Fire Alarm, dtype: int64

In [5]:
df.isna().sum()
#data set has no missing values

Unnamed: 0        0
UTC               0
Temperature[C]    0
Humidity[%]       0
TVOC[ppb]         0
eCO2[ppm]         0
Raw H2            0
Raw Ethanol       0
Pressure[hPa]     0
PM1.0             0
PM2.5             0
NC0.5             0
NC1.0             0
NC2.5             0
CNT               0
Fire Alarm        0
dtype: int64

In [6]:
# Remove missing values here, before we split it into X and y?
X = df.drop(["Unnamed: 0", "UTC", "CNT", "Fire Alarm"], axis=1)
y = df["Fire Alarm"]

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn import tree
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split as tts
z=5

# Need pipelines for:
# - Logistics regression
# - KNN classifier
# - Decision tree (see below)
scaler = RobustScaler()
pipe_dt = Pipeline([
    ("scaler", scaler), # switch out for another thing?
    ("classify", tree.DecisionTreeClassifier())
])

X_train_dt, X_test_dt, y_train_dt, y_test_dt = tts(X, y, test_size=0.20, random_state=42, stratify=y) 

pg_dt = {'classify__min_samples_split': [2,10,50,100,250,600]} 
clf_dt = GridSearchCV(pipe_dt, param_grid =pg_dt,cv=z, return_train_score=True)
clf_dt.fit(X_train_dt,y_train_dt)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', RobustScaler()),
                                       ('classify', DecisionTreeClassifier())]),
             param_grid={'classify__min_samples_split': [2, 10, 50, 100, 250,
                                                         600]},
             return_train_score=True)

In [34]:
from sklearn.neighbors import KNeighborsClassifier

z=5

pipe_knn = Pipeline([
    ("scaler", scaler), # switch out for another thing?
    ("classify", KNeighborsClassifier()) # What params?
])

X_train_knn, X_test_knn, y_train_knn, y_test_knn = tts(X, y, test_size=0.20, random_state=42, stratify=y) 
parameter={"classify__n_neighbors":[10,20,50,100,200]}
knncv=GridSearchCV(pipe_knn,parameter,cv=z, return_train_score=True)
knncv.fit(X_train_knn,y_train_knn)

In [ ]:
from sklearn.linear_model import LogisticRegression

z=5

pipe_lg = Pipeline([
    ("scaler", scaler), # switch out for another thing?
    ("classify", LogisticRegression()) # What params?
])

X_train_lg, X_test_lg, y_train_lg, y_test_lg = tts(X, y, test_size=0.20, random_state=42, stratify=y)

pg_lg = {"classify__C":[0.5,0.01,1,2,3,4,5], 'classify__max_iter':[1,2,3,4,5,10,50]} 
clf_lg = GridSearchCV(pipe_lg, param_grid =pg_lg,cv=z, return_train_score=True)
clf_lg.fit(X_train_lg,y_train_lg)

c:\Users\Hildur\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Hildur\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/st

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('classify', LogisticRegression())]),
             param_grid={'classify__C': [0.5, 0.01, 1, 2, 3, 4, 5],
                         'classify__max_iter': [1, 2, 3, 4, 5, 10, 50]},
             return_train_score=True)